In [59]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression

### Train Classification Model
Classes: 0, 1, 2, 3 - relevance score provided by dataset
1. Sample as dataset favours one class
2. Train simple multi-class classification
3. Evaluate accuracy, precision, recall, F1 SCore, AUC
4. Save model to evaluate against test set

#### Create dataset to use for training
1. Start with X = cosine similarity and y = relvance score
2. Later add more features to X (code and docstring embeddings, results from static analysis?)

In [23]:
df = pd.read_csv('data/sim.csv', usecols=['sim', 'relevance'])
df = df.dropna()
df.head()

,relevance,sim
0,2,0.685977
1,2,0.469626
2,2,0.490250
3,3,0.568199
4,3,0.166293


In [58]:
X = df.sim.values.reshape(-1, 1)
Y = df.relevance.values.reshape(-1, 1)
X[:5]

array([[0.68597656],
       [0.46962649],
       [0.49024951],
       [0.56819922],
       [0.16629307]])

#### Sample dataset to balance classes

In [55]:
smote = SMOTE()
X_smote, Y_smote = smote.fit_resample(X, Y)

Counter(Y_smote)

Counter({2: 112, 3: 112, 0: 112, 1: 112})

#### Train model

In [60]:
clf = LogisticRegression(multi_class='ovr')

clf.fit(X_smote, Y_smote)

LogisticRegression(multi_class='ovr')